# File to generate SWOT OSSE tracks 

In [1]:
import numpy as np 
import xarray as xr
import glob 
import os 
from scipy import interpolate
import random
import datetime

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy

import gc

from joblib import Parallel
from joblib import delayed as jb_delayed


Refer the satellite type data in the variable *sat_type*, either **wide_swath** for swath measurements, or **nadir** for nadir measurements. 

In [2]:
sat_type = "nadir"
# sat_type = "wide_swath"

Choice of the OSSE zone between *crossover_CCS*, *crossover_centerpacific* and *crossover_hawaii*.

In [3]:
### ZONE OF CROSSOVERS ###

#zone = "crossover_CCS"
#zone = "crossover_centerpacific"
#zone = "crossover_hawaii"

### ZONE OF IMAGINARY CROSSOVERS ### 

zone = "hawaii"

Choice of the variables to integrate in the interpolated measurements among : 
- *ssh_cor* : Total DAC corrected SSH, renamed *ssh* in the output dataset 
- *ssh_bm* : Balanced Motion,
- *ssh_it1* : 1st Mode Internal Tide, 
- *ssh_it2* : 2nd Mode Internal Tide, 
- *ssh_it3* : 3d Mode Internal Tide, 
- *ssh_it_tot* : Total Internal Tide,
- *ssh_it1_coherent* : Coherent part of the 1st Mode Internal Tide. 

In [4]:
list_variables = ["ssh_cor","ssh_bm","ssh_it1"]
dic_attrs = dict({"ssh":"Total DAC corrected SSH.",
                  "ssh_bm":"Balanced Motion.",
                  "ssh_it1":"1st Mode Internal Tide.",
                  "ssh_it2":"2nd Mode Internal Tide.",
                  "ssh_it3":"3d Mode Internal Tide.",
                  "ssh_it_tot":"Total Internal Tide."})

Definition of the coordinates limits : 

In [5]:
if zone == "crossover_CCS":
    lon_min = 225; lon_max = 245;lat_min = 25; lat_max = 45;
elif zone == "crossover_centerpacific":
    lon_min = 200; lon_max = 220;lat_min = 25; lat_max = 45;
elif zone == "crossover_hawaii":
    lon_min = 180; lon_max = 200;lat_min = 25; lat_max = 45;
elif zone == "hawaii":
    lon_min = 185; lon_max = 205;lat_min = 15; lat_max = 35;

## 1. - Functions  

In [6]:
def open_model_data(date,var,omit_next_step = False):

    """
    Opens the model field and create the RegularGridInterpolator

    ARGS : 
    date (np.datetime64) - the date 
    var (str) - the variable name
    omit_next_step (bool) - if the next step (midnight next day) needs to be omited

    """


    if "ssh_it" in var : # variable of internal tide ssh, contained inside MITgcm_it directory 
        dir_input_ssh = f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/"
        if omit_next_step:
            path = os.path.join(dir_input_ssh,"MITgcm_it_"+date.astype('str').replace("-","")+".nc")
        else : 
            path = []
            path.append(os.path.join(dir_input_ssh,"MITgcm_it_"+date.astype('str').replace("-","")+".nc"))
            path.append(os.path.join(dir_input_ssh,"MITgcm_it_"+(date+np.timedelta64(1,"D")).astype('str').replace("-","")+".nc"))
    else : # other variable contained inside MITgcm_filtered_final directory 
        dir_input_ssh = f"/bettik/bellemva/MITgcm/MITgcm_filtered_final"
        if omit_next_step:
            path = os.path.join(dir_input_ssh,"MITgcm_filt_"+date.astype('str').replace("-","")+".nc")
        else : 
            path = []
            path.append(os.path.join(dir_input_ssh,"MITgcm_filt_"+date.astype('str').replace("-","")+".nc"))
            path.append(os.path.join(dir_input_ssh,"MITgcm_filt_"+(date+np.timedelta64(1,"D")).astype('str').replace("-","")+".nc"))

    if omit_next_step:
        ds_ssh = xr.open_dataset(path)
    else : 
        ds_ssh = xr.open_mfdataset(path,combine="nested",concat_dim="time")
    
    ds_ssh = ds_ssh.sel(longitude = slice(lon_min,lon_max),latitude=slice(lat_min,lat_max),drop=True)
    
    if omit_next_step:
        finterp = interpolate.RegularGridInterpolator([ds_ssh.time[0:24].values.astype("float64"),ds_ssh.latitude.values,ds_ssh.longitude.values],
                                                ds_ssh[var][0:24].values,
                                                bounds_error=False)
    else : 
        finterp = interpolate.RegularGridInterpolator([ds_ssh.time[0:25].values.astype("float64"),ds_ssh.latitude.values,ds_ssh.longitude.values],
                                                ds_ssh[var][0:25].values,
                                                bounds_error=False)
    
    return finterp 

In [7]:
def interp_satellite(latitude_array,longitude_array,time_array,time_serie,interp,sat_type,var):
    
    """
    This function interpolates the modelled ssh on the provided swaths (for SWOT satellite). 

        - latitude_data_array : data array containing the latitude information of each pixel (shape = [num_lines,num_pixels])
        - longitude_data_array : data array containing the longitude information of each pixel (shape = [num_lines,num_pixels])
        - time_data_array : data array containing the time information of each pixel (shape = [num_lines,num_pixels])
        - time_serie : initial time_array of the observations (before converting to float)
        - interp : interpolator as provided by the function open_model_data

    Returns : 
        - ds : dataset of interpolated ssh. Longitude and latitude are coodinates indexed by [num_lines,num_pixels]. Time and ssh_cor are variables. 

    """


    points = np.concatenate((np.expand_dims(time_array.flatten(),axis=1),
                                       np.expand_dims(latitude_array.flatten(),axis=1),
                                       np.expand_dims(longitude_array.flatten(),axis=1)),axis=1)
    
    ssh_interp = interp(points).reshape(time_array.shape)

    #if var is "ssh_cor", renaming it into "ssh" in the netcdf file 
    if var == "ssh_cor":
        var = "ssh"

    if sat_type == "wide_swath":

        ds = xr.Dataset({'time':(('num_lines'),time_serie),
                        var:(['num_lines','num_pixels'],ssh_interp),}, 
                        coords={'latitude':(['num_lines','num_pixels'],latitude_array),
                                'longitude':(['num_lines','num_pixels'],longitude_array)} 
                        )
        
    elif sat_type == "nadir":

        ds = xr.Dataset({var:('time',ssh_interp.T)}, 
                    coords={'time':('time',time_serie),
                            'latitude':('time',latitude_array),
                            'longitude':('time',longitude_array)} 
                    )

    else : 
        print("Variable sat_type is not valid.")

    # Setting the variable attributes 
    ds[var] = ds[var].assign_attrs({"description":dic_attrs[var],
                           "units":"[m]"})
    
    return ds 


## 2. - Data loading 

The information of SWOT swaths coordinates are dowloaded. The file contain the North Pacific region. 

In [8]:
ds = xr.open_dataset("/bettik/bellemva/satellite_track/swot/track_swot_pacific.nc")

if sat_type == "nadir" :
    ds = ds.isel(num_pixels=35)

/home/bellemva/miniconda3/envs/pangeo-forge/lib/python3.10/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


The file is cropped to only keep the region of interest : 

In [9]:
if "longitude_nadir" in ds.keys() and "latitude_nadir" in ds.keys():
    lon_nad = ds.longitude_nadir
    lat_nad = ds.latitude_nadir
else : 
    lon_nad = ds.longitude
    lat_nad = ds.latitude

### if zone == "hawaii", shifting the crossover coordinates to hawaii ###
if zone=="hawaii":
    # sleecting data from another crossover
    lon_min_bis = 225; lon_max_bis = 245;lat_min_bis = 25; lat_max_bis = 45;
    mask = (lon_nad>lon_min_bis)*(lon_nad<lon_max_bis)*(lat_nad>lat_min_bis)*(lat_nad<lat_max_bis)
    
    #shifting coordinates 
    ds = ds.where(mask,drop=True)
    # ds = ds.where(mask)
    ds = ds.assign_coords({"longitude" : ds.longitude - (lon_min_bis - lon_min)})
    ds = ds.assign_coords({"latitude" : ds.latitude - (lat_min_bis - lat_min)})

### else : ### 
else : 
    mask = (lon_nad>lon_min)*(lon_nad<lon_max)*(lat_nad>lat_min)*(lat_nad<lat_max)
    ds = ds.where(mask,drop=True)

A mask is applied to select the measures with across track distance between 10km and 60km, if wide swath type is interpolated. 

In [10]:
if sat_type == "wide_swath" :
    mask_act = np.logical_and(np.abs(ds.cross_track_distance)>10000,
                            np.abs(ds.cross_track_distance)<60000)

    # ds_m = ds.where(mask_act,drop=True)

    # ds_m = ds.copy(deep=True)
    # ds_m["longitude"] = ds["longitude"].where(mask_act)
    # ds_m["latitude"] = ds["latitude"].where(mask_act)
    # ds_m['time'] = ds.time
    # ds = ds_m
    # del ds_m

The SWOT coordinates for the full period are computed from the SWOT coordinates dataset. 

In [11]:
### CREATION OF THE TIME DATASET COOORDINATE ### 

time_dataset = np.empty((93*ds.time.values.shape[0],),dtype="datetime64[ns]")
time_delta = (ds.time.values-ds.time.values[0])
shift = np.timedelta64(int(0.99349*24*3600*1E9),"ns")

for i in range (93) : # number of swot cycles computed, in the 91 days of experiment 
    time_dataset[i*ds.time.values.shape[0]:(i+1)*ds.time.values.shape[0]]=np.datetime64("2012-05-01")+time_delta+i*shift

### CREATION OF THE LONGITUDE AND LATITUDE DATASET COOORDINATE ### 
longitude_dataset = np.tile(ds.longitude.values.T,reps=93).T
latitude_dataset = np.tile(ds.latitude.values.T,reps=93).T
if sat_type == "wide_swath" :
    mask_act_dataset = np.tile(mask_act.values.T,reps=93).T

## 3. - Track interpolating

In [12]:
name_folder = dict({"crossover_CCS":"2023b_SSHmapping_HF_Crossover_California","crossover_centerpacific":"2023c_SSHmapping_HF_Crossover_CenterPacific",\
                    "crossover_hawaii":"2023d_SSHmapping_HF_Crossover_Hawaii","hawaii":"2023e_SSHmapping_HF_Hawaii_new"}) # folder info for each of the zones
name_subfolder = dict({"nadir":"dc_obs_nadirs/swot/","wide_swath":"dc_obs_swot/"}) # subfolder info for each of the sat types
name_file = dict({"nadir":"NADIR","wide_swath":"SWOT"}) # filename info for each of the sat types

def create_dataset(d,sat_type,omit_next_step=False):

    ########## NEW DEV ##########
    mask = ((time_dataset>=d)*(time_dataset<d+np.timedelta64(1,"D")))
    time_array = time_dataset[mask]
    time_array_float = time_array.astype("float64")
    if sat_type == "wide_swath":
        longitude_array = longitude_dataset[mask,:]
        latitude_array = latitude_dataset[mask,:]
    elif sat_type == "nadir":
        longitude_array = longitude_dataset[mask]
        latitude_array = latitude_dataset[mask]
    #############################
    
    # expanding time array across swath  
    if sat_type == "wide_swath":
        time_array_float = np.repeat(np.expand_dims(time_array_float,axis=1),axis=1,repeats=69)
        
    #elif sat_type == "nadir":
    #    time_data_array = ds.assign(time=("num_lines",time_array_float)).time

    # Interpolating for each variable
    lst_ds_interp = []
    for var in list_variables :  
        finterp = open_model_data(d,var,omit_next_step)
        lst_ds_interp.append(interp_satellite(latitude_array = latitude_array,longitude_array = longitude_array,
                                                time_array = time_array_float,time_serie = time_array,
                                                interp = finterp,sat_type=sat_type,var=var))
    
    # Merging each variable file 
    ds_interp = xr.merge(lst_ds_interp)

    if sat_type == "wide_swath" :
        for var in list_variables : 
            if var == "ssh_cor":
                var = "ssh" 
            ds_interp[var] = ds_interp[var].where(mask_act_dataset[mask],np.nan)

    # Saves the observation  
    ds_interp.to_netcdf(f"/bettik/bellemva/ocean_data_challenge/{name_folder[zone]}/{name_subfolder[sat_type]}SSH_{name_file[sat_type]}_"+d.astype('str')+".nc",
                            encoding = {"time" : {'units' : "nanoseconds since 2012-01-01 00:00:00.000000000"}})

    del ds_interp, finterp, time_array
    gc.collect()
    
    print(d,"done")

This loop on dates calls the function **interp_satellite**. 

In [13]:
dates = np.arange(np.datetime64("2012-05-02"),np.datetime64("2012-08-01"))
# dates = np.arange(np.datetime64("2012-07-30"),np.datetime64("2012-08-01"))

for d in dates:
    omit_next_step=False 
    if d==np.datetime64("2012-07-31"):
        omit_next_step=True
    create_dataset(d,sat_type,omit_next_step=omit_next_step)


2012-05-02 done
2012-05-03 done
2012-05-04 done
2012-05-05 done
2012-05-06 done
2012-05-07 done
2012-05-08 done
2012-05-09 done
2012-05-10 done
2012-05-11 done
2012-05-12 done
2012-05-13 done
2012-05-14 done
2012-05-15 done
2012-05-16 done
2012-05-17 done
2012-05-18 done
2012-05-19 done
2012-05-20 done
2012-05-21 done
2012-05-22 done
2012-05-23 done
2012-05-24 done
2012-05-25 done
2012-05-26 done
2012-05-27 done
2012-05-28 done
2012-05-29 done
2012-05-30 done
2012-05-31 done
2012-06-01 done
2012-06-02 done
2012-06-03 done
2012-06-04 done
2012-06-05 done
2012-06-06 done
2012-06-07 done
2012-06-08 done
2012-06-09 done
2012-06-10 done
2012-06-11 done
2012-06-12 done
2012-06-13 done
2012-06-14 done
2012-06-15 done
2012-06-16 done
2012-06-17 done
2012-06-18 done
2012-06-19 done
2012-06-20 done
2012-06-21 done
2012-06-22 done
2012-06-23 done
2012-06-24 done
2012-06-25 done
2012-06-26 done
2012-06-27 done
2012-06-28 done
2012-06-29 done
2012-06-30 done
2012-07-01 done
2012-07-02 done
2012-07-

In [ ]:
create_dataset(np.datetime64("2012-05-02"),sat_type)

## DEV


## 3. - Plots 

In [ ]:
def map_ssh(ssh,mask,ax,vmin,vmax): 
    """
    This function plots the data of Nadirs. 
    """
    return ax.scatter(ssh.longitude.values[mask],
            ssh.latitude.values[mask],
            c=ssh.ssh_cor.values[mask],
            cmap="Spectral",
            s=0.1,
            vmin = vmin,
            vmax = vmax
            )

In [ ]:
def map_ssh_swot(ssh,ax,vmin,vmax):
    """
    This function plots the data of SWOT. 
    """
    return ax.scatter(ssh.longitude.values,
            ssh.latitude.values,
            c=ssh.ssh_cor.values,
            cmap="Spectral",
            s=0.1,
            vmin = vmin,
            vmax = vmax
            )

In [ ]:

def plot(ssh_alg,ssh_j3,ssh_s3a,ssh_s3b,ssh_c2,ssh_swot,
         mask_alg,mask_j3,mask_s3a,mask_s3b,mask_c2,date): 
    
    """
    This function creates the plots of the modelled satellite tracks for a given date. 

    Input : 
        - ssh_alg, ssh_j3,ssh_s3a,ssh_s3b,ssh_c2,ssh_swot : datasets of interpolated ssh 
        - mask_alg,mask_j3,mask_s3a,mask_s3b,mask_c2 : mask containing True when the point is measured at time corresponding to date input parameter 
        - date 

    Returns : None (but saves output)
    
    """
    

    fig, ax = plt.subplots(figsize=(9,4),dpi=200,subplot_kw={"projection": ccrs.PlateCarree(central_longitude=0.)})

    ax.set_extent([-145, -115,15, 45],crs=ccrs.PlateCarree())
    vmin=-1
    vmax=1

    for ssh,mask in zip((ssh_alg,ssh_j3,ssh_s3a,ssh_s3b,ssh_c2),
                        (mask_alg,mask_j3,mask_s3a,mask_s3b,mask_c2)):
        if mask.sum()>0:

            map_ssh(ssh,mask,ax,vmin=vmin,vmax=vmax)

    if ssh_swot is not None : 
        map_swot = map_ssh_swot(ssh_swot,ax,vmin=vmin,vmax=vmax)


    ax.add_feature(cfeature.LAND,color="lightgrey",zorder=1)
    ax.add_feature(cfeature.OCEAN,color="black",zorder=0)
    ax.coastlines(lw=0.5)
    ax.set_aspect("equal")
    ax.set_title("SSH Observations")

    gridlines = ax.gridlines(
        alpha=0.3,
        color="white",
        draw_labels=True,
        xlocs=np.arange(-180,-115,10),
        ylocs=np.arange(10,45,5),
        # xformatter=cartopy.mpl.ticker.LongitudeFormatter(zero_direction_label=True,),
    )
    gridlines.right_labels = False
    gridlines.top_labels  = False
    gridlines.xlabel_style = {'size': 11}
    gridlines.ylabel_style = {'size': 11}
    ax.set_aspect("equal")

    #cbar = plt.colorbar(mappable=map_swot)
    #cbar.set_label("SSH")

    ax.text(s=date.astype('str'),x=-123.5,y=44,c="black",fontsize=8)

    plt.savefig("/home/bellemva/CCS/data/interp_satellite_track/frames/"+date.astype("str")+".png")


In [ ]:
time_array = np.arange(np.datetime64("2012-06-01"),np.datetime64("2012-06-14"))

for d in time_array : 
    # Imports the datasets of SSH 
    ds_alg = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/dc_nadir/alg/ds_alg_"+d.astype("str").replace("-","")+".nc")
    ds_s3a = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/dc_nadir/s3a/ds_s3a_"+d.astype("str").replace("-","")+".nc")
    ds_s3b = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/dc_nadir/s3b/ds_s3b_"+d.astype("str").replace("-","")+".nc")
    ds_c2 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/dc_nadir/c2/ds_c2_"+d.astype("str").replace("-","")+".nc")
    ds_j3 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/dc_nadir/j3/ds_j3_"+d.astype("str").replace("-","")+".nc")
    ds_swot = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/dc_swot/ds_swot_"+d.astype("str").replace("-","")+".nc")

    for i in range (24):
        time = d+np.timedelta64(i,"h")
        # Creates the mask selecting data for one given date 
        mask_alg = ds_alg.time.astype('datetime64[h]')==time
        mask_s3a = ds_s3a.time.astype('datetime64[h]')==time
        mask_s3b = ds_s3b.time.astype('datetime64[h]')==time
        mask_c2 = ds_c2.time.astype('datetime64[h]')==time
        mask_j3 = ds_j3.time.astype('datetime64[h]')==time

        # Creates the SWOT ssh dataset 

        if i==6 : 
            ds_swot_selec = ds_swot.isel(num_lines = slice(0,2200))
        elif i==23 : 
            ds_swot_selec =ds_swot.isel(num_lines = slice(2200,4350))
        else : 
            ds_swot_selec=None 

        plot(ssh_alg=ds_alg,ssh_j3=ds_j3,ssh_s3a=ds_s3a,ssh_s3b=ds_s3b,ssh_c2 = ds_c2,ssh_swot=ds_swot_selec,
             mask_alg=mask_alg,mask_j3=mask_j3,mask_s3a=mask_s3a,mask_s3b=mask_s3b,mask_c2=mask_c2,date=time)    
        

    

In [ ]:
dates = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-06-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/interp_satellite_track/file.sh","w")
file.close()

for d in dates:

    command = "ncap2 -A -s \"ssh_10=ssh_cor*0+10\" "

    command += "/bettik/bellemva/2022a_CCS_CalVal/dc_swot/ds_swot_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/2022a_CCS_CalVal/dc_swot/ds_swot_"+d.astype('str').replace("-","")+".nc \n"

    file = open("/home/bellemva/CCS/data/interp_satellite_track/file.sh","a")
    
    file.write(command)

    file.close()

In [ ]:
ds_alg

In [ ]:
ds_alg = xr.open_mfdataset(["/bettik/bellemva/2022a_CCS_CalVal/dc_nadir/alg/ds_alg_20120604.nc",
                            "/bettik/bellemva/2022a_CCS_CalVal/dc_nadir/alg/ds_alg_20120605.nc"],combine='nested',concat_dim='time')

### Plot for Liège poster

In [ ]:
#ds_alg = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/alg/ds_alg_20120604.nc")
ds_alg2 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/alg/ds_alg_20120603.nc")

#ds_s3a = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/s3a/ds_s3a_20120604.nc")
ds_s3a2 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/s3a/ds_s3a_20120603.nc")

#ds_s3b = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/s3b/ds_s3b_20120604.nc")
ds_s3b2 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/s3b/ds_s3b_20120603.nc")

#ds_c2 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/c2/ds_c2_20120604.nc")
ds_c22 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/c2/ds_c2_20120603.nc")

#ds_j3 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/j3/ds_j3_20120604.nc")
ds_j32 = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/nadir/j3/ds_j3_20120603.nc")

ds_swot = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/swot/ds_swot_20120603.nc")

fig, ax = plt.subplots(figsize=(9,4),dpi=200,subplot_kw={"projection": ccrs.PlateCarree(central_longitude=0.)})

ax.set_extent([-130, -120,30, 40],crs=ccrs.PlateCarree())
vmin=0.2
vmax=0.6

#for ssh in (ds_alg,ds_alg2,ds_j3,ds_j32,ds_s3a,ds_s3a2,ds_s3b,ds_s3b2,ds_c2,ds_c22):
for ssh in (ds_alg2,ds_j32,ds_s3a2,ds_s3b2,ds_c22):
    #mask = np.ones(len(ssh.longitude.values),dtype='int')
    #map_ssh(ssh,mask,ax,vmin=vmin,vmax=vmax)

    ax.scatter(ssh.longitude.values,
           ssh.latitude.values,
           c=ssh.ssh_cor.values,
            cmap="Spectral",
            s=0.5,
            vmin = vmin,
            vmax = vmax
            )


map_swot = map_ssh_swot(ds_swot,ax,vmin=vmin,vmax=vmax)

ax.add_feature(cfeature.LAND,color="lightgrey",zorder=1)
ax.add_feature(cfeature.OCEAN,color="black",zorder=0)
ax.coastlines(lw=0.5)
ax.set_aspect("equal")
ax.set_title("Sea Surface Height (SSH) Observations")

gridlines = ax.gridlines(
    alpha=0.5,
    color="white",
    draw_labels=True,
    xlocs=np.arange(-180,-115,2.5),
    ylocs=np.arange(10,45,2.5),
    # xformatter=cartopy.mpl.ticker.LongitudeFormatter(zero_direction_label=True,),
)
gridlines.right_labels = False
gridlines.top_labels  = False
gridlines.xlabel_style = {'size': 11}
gridlines.ylabel_style = {'size': 11}
ax.set_aspect("equal")

cbar = plt.colorbar(mappable=map_swot,)
cbar.set_label("SSH [m]")

#ax.text(s=date.astype('str'),x=-123.5,y=44,c="black",fontsize=8)

# plt.savefig("/home/bellemva/CCS/data/interp_satellite_track/frames/"+date.astype("str")+".png")

In [ ]:
plt.hist(ds_swot.ssh_cor.values.flatten())

## 4. - Formatting 

This cell aims at formatting the SWOT data prior to its interpolation. During interpolation phase, gap between two swaths hasn't been masked by NaNs. This processing step will mask the data around Nadir according tot the mask on SWOT real Data. 

In [ ]:
ds_swot = xr.open_dataset("/bettik/bellemva/2022a_CCS_CalVal/swot/ds_swot_20120603.nc")

In [ ]:
# mask of measured values, taken from SWOT real data 
mask_line = np.array([ True,  True, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False, False,False, False, False, False, False, False,  True,  True,  True,False, False, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False, False,False, False, False, False,  True,  True])
# transfering it into 2d data array 
mask_array = np.repeat(np.expand_dims(mask_line,axis=0),axis=0,repeats=ds_swot.dims['num_lines'])

In [ ]:
# function to save the masked data for a given date 
def save_masked_data(name_file,mask_array):
    ds_swot = xr.open_dataset(name_file)
    ds_swot.ssh_cor.values[mask_array]=np.nan
    ds_swot.to_netcdf(name_file.replace("swot/","swot_bis/"))

In [ ]:
# list of filenames 
list_file = !ls /bettik/bellemva/2022a_CCS_CalVal/swot/ds*.nc

In [ ]:
# parallel computing
res = np.array(Parallel(n_jobs=6,backend='multiprocessing')(jb_delayed(save_masked_data)(list_file[i],mask_array) for i in range(len(list_file))))

## 5. - Comparison with SWOT real data (to debug interpolation problem)

In [ ]:
import xarray as xr 

In [ ]:
ssh_swot_real = xr.open_dataset("/bettik/bellemva/obs_294_305_32_42_21600_SWOT_SSH_20231029_06h00_addmdt.nc").load()


In [ ]:
ssh_swot_model = xr.open_dataset("/bettik/bellemva/ocean_data_challenge/2023e_SSHmapping_HF_Hawaii/dc_obs_swot/SSH_SWOT_2012-06-10.nc")

In [ ]:
ssh_swot_new = xr.open_dataset("/bettik/bellemva/ocean_data_challenge/2023e_SSHmapping_HF_Hawaii_new/dc_obs_swot/SSH_SWOT_2012-06-01.nc")
ssh_swot_new

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.plot(np.arange(ssh_swot_model.dims["num_pixels"]),ssh_swot_model.ssh_it1[100,:])
plt.scatter(np.arange(ssh_swot_model.dims["num_pixels"]),np.isnan(ssh_swot_model.ssh_it1[100,:]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.plot(np.arange(ssh_swot_real.dims["num_pixels"]),ssh_swot_real.SSH[100,:])
plt.plot(np.arange(ssh_swot_real.dims["num_pixels"]),ssh_swot_real.longitude[100,:]/ssh_swot_real.longitude[100,:],c='red')

plt.scatter(np.arange(ssh_swot_real.dims["num_pixels"]),np.isnan(ssh_swot_real.SSH[100,:]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.plot(np.arange(ssh_swot_new.dims["num_pixels"]),ssh_swot_new.ssh[100,:])
plt.plot(np.arange(ssh_swot_new.dims["num_pixels"]),ssh_swot_new.longitude[100,:]/ssh_swot_new.longitude[100,:],c='red')

plt.scatter(np.arange(ssh_swot_new.dims["num_pixels"]),np.isnan(ssh_swot_new.ssh[100,:]))

In [ ]:
lon = np.arange(190,200,1/20)
lat = np.arange(15,35,1/20)
lon2d,lat2d = np.meshgrid(lon,lat)

In [ ]:
longitudes = ssh_swot_new.longitude.values[0:1100,:]
latitudes = ssh_swot_new.latitude.values[0:1100,:]
values = ssh_swot_new.ssh_it1.values[0:1100,:]

In [ ]:
from scipy.interpolate import griddata 
ssh_interp = griddata(points=(longitudes.ravel(), latitudes.ravel()),
                      values = values.ravel(),
                      xi=(lon2d.ravel(),lat2d.ravel()))

In [ ]:
plt.pcolormesh(ssh_interp.reshape(lon2d.shape))

In [ ]:
longitudes_model = ssh_swot_model.longitude.values[0:1100,:]
latitudes_model = ssh_swot_model.latitude.values[0:1100,:]
values_model = ssh_swot_model.ssh_it1.values[0:1100,:]

In [ ]:
ssh_interp_model = griddata(points=(longitudes_model.ravel(), latitudes_model.ravel()),
                      values = values_model.ravel(),
                      xi=(lon2d.ravel(),lat2d.ravel()))

In [ ]:
plt.pcolormesh(ssh_interp_model.reshape(lon2d.shape))